### Imports

In [1]:
import pandas as pd
from pprint import pprint
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime

In [2]:
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/opt/homebrew/Caskroom/miniconda/base/envs/cs234_final_project/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import sys
sys.path.append("/Users/sahiljain/Documents/CS234-Spring 24/final-project/replication")

In [4]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

### Load data

In [5]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-03-01'

In [6]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (97013, 8)


In [8]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,6.461975,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,41.200775,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,33.090427,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,43.777557,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,40.190235,7325600,CAT,0


In [9]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


In [10]:
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,7.622500,7.660714,7.585000,6.461975,493729600,AAPL,0,0.0,6.483364,6.45176,100.0,66.666667,100.0,6.461975,6.461975,20.040001,0.0
1,2010-01-04,56.630001,57.869999,56.560001,41.200775,5277400,AMGN,0,0.0,6.483364,6.45176,100.0,66.666667,100.0,41.200775,41.200775,20.040001,0.0
2,2010-01-04,40.810001,41.099998,40.389999,33.090427,6894300,AXP,0,0.0,6.483364,6.45176,100.0,66.666667,100.0,33.090427,33.090427,20.040001,0.0
3,2010-01-04,55.720001,56.389999,54.799999,43.777557,6186700,BA,0,0.0,6.483364,6.45176,100.0,66.666667,100.0,43.777557,43.777557,20.040001,0.0
4,2010-01-04,57.650002,59.189999,57.509998,40.190235,7325600,CAT,0,0.0,6.483364,6.45176,100.0,66.666667,100.0,40.190235,40.190235,20.040001,0.0
